# Тестирование произвольного классификатора

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Зададим нужные константы для тестирования:

In [2]:
# we perform T*Q-fold cross validation (T*Q-fold CV)
T = 3  # number of performing cross validations
Q = 5  # number of folds

# Select a classifier
PATH_TO_CLASSIFIER = '../results/boosting/boosting_classifier.pkl'
# Select features for testing classifier
# BEST_FEATURES_KNN = [
#     1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 16, 20,
#     21, 22, 23, 26, 27, 30, 31, 32, 34, 36, 37, 38, 39, 40,
#     41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60,
#     61, 62, 63, 65, 66, 67, 68, 71, 72, 74, 76, 77, 78, 79, 80,
#     81, 83, 84, 86, 88, 89, 91, 92, 93, 94, 95
# ]

Загрузим подготовленные данные:

In [15]:
from sklearn.externals import joblib
X_train = joblib.load('../data/X_train.pkl')
X_test = joblib.load('../data/X_test.pkl')
y_train = joblib.load('../data/y_train.pkl')
y_test = joblib.load('../data/y_test.pkl')

# X_train = X_train.iloc[:, BEST_FEATURES_KNN]
# X_test = X_test.iloc[:, BEST_FEATURES_KNN]

#display(X_train)
# display(list(X_test.columns.values))
# display(y_test)
# display(y_train)
# sum(y_test)
X_train_aux = X_train[X_train['DOSStub/entropy'] > 0.3333]
X_train_aux = X_train_aux[X_train_aux['DOSStub/entropy'] < 0.3334]



X_train_aux

,DIRECTORY_ENTRY_BOUND_IMPORT/exists,DIRECTORY_ENTRY_DEBUG/exists,DIRECTORY_ENTRY_DELAY_IMPORT/exists,DIRECTORY_ENTRY_EXPORT/exists,DIRECTORY_ENTRY_IMPORT/exists,DIRECTORY_ENTRY_RESOURCE/exists,DIRECTORY_ENTRY_TLS/exists,DOSStub/entropy,DOSStub/zeros,DOS_HEADER.e_cblp,...,SECTIONS/entry_point_in_last_section,SECTIONS/entry_point_in_writeable_section,SECTIONS/executable_sections,SECTIONS/max_section_entropy,SECTIONS/rsrc_section_entropy,SECTIONS/sections_with_zero_entropy_count,SECTIONS/text_section_entropy,SECTIONS/unusual_section_names,SECTIONS/writeable_and_executable_sections,SECTIONS/writeable_sections
29258,-0.145786,-0.079078,-0.15013,-0.304439,-0.078087,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.268217,-0.186286,-0.665014,0.581034,-0.460900,-0.288560,-0.094187
54395,-0.145786,-0.079078,-0.15013,-0.304439,0.020719,-2.994258,-0.507212,0.333303,-0.542699,-0.011388,...,4.395228,-0.452174,0.569053,0.459248,-2.024843,0.258948,0.809948,0.131280,0.518246,1.028014
16372,-0.145786,-0.079078,-0.15013,-0.304439,0.020719,-2.994258,-0.507212,0.333303,-0.542699,-0.011388,...,4.395228,-0.452174,0.569053,0.471684,-2.024843,0.258948,0.813861,0.131280,0.518246,1.028014
32336,-0.145786,-0.079078,-0.15013,-0.304439,-0.473313,-2.994258,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,2.211540,1.407226,1.496194,-2.024843,1.182910,-1.921663,1.315639,2.131858,0.466914
52583,-0.145786,-0.079078,-0.15013,-0.304439,-0.275700,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.335079,-0.519568,-0.665014,0.394407,-0.460900,-0.288560,1.028014
29950,-0.145786,0.129710,-0.15013,-0.304439,-0.473313,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,-0.452174,-0.269120,-0.441539,0.195015,-0.665014,0.526495,-0.460900,-0.288560,-0.655287
52581,-0.145786,-0.079078,-0.15013,-0.304439,-0.374507,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,2.211540,-0.269120,-0.835379,0.749508,0.258948,0.402564,-0.460900,0.518246,1.589115
27396,-0.145786,0.129710,-0.15013,-0.304439,-0.670927,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,-0.452174,-0.269120,-1.913815,-0.159834,-0.665014,0.063210,-0.460900,-0.288560,-1.216388
12945,-0.145786,-0.079078,-0.15013,-0.304439,0.020719,-2.994258,-0.507212,0.333303,-0.542699,-0.011388,...,4.395228,-0.452174,0.569053,0.526626,-2.024843,0.258948,0.831150,0.131280,0.518246,1.028014
52227,-0.145786,-0.079078,-0.15013,-0.304439,-0.374507,0.333973,-0.507212,0.333303,-0.542699,-0.011388,...,-0.227519,2.211540,-0.269120,-0.840400,0.776478,0.258948,0.243522,-0.460900,0.518246,1.589115


Тестирование классификатора KNeighborsClassifier.
Получим 4 датафрейма:
* tns_df --- статистика о true negatives (файл: clear, предсказание: clear)
* fps_df --- статистика о false positives (файл: clear, предсказание: malware)
* fns_df --- статистика о false negatives (файл: malware, предсказание: clear)
* tps_df --- статистика о true positives (файл: malware, предсказание: malware)

In [4]:
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import confusion_matrix


def test_classifier(classifier, X, y):
    tns, fps, fns, tps = ([], [], [], [])

    rkf = RepeatedKFold(n_splits=Q, n_repeats=T, random_state=42)
    for train_index, test_index in rkf.split(X):
        curr_X_train, curr_X_test = X.iloc[train_index], X.iloc[test_index]
        curr_y_train, curr_y_test = y.iloc[train_index], y.iloc[test_index]
        classifier.fit(curr_X_train, curr_y_train)
        print('*** fitted ***')
        curr_y_pred = classifier.predict(curr_X_test)
        curr_tn, curr_fp, curr_fn, curr_tp = (
            confusion_matrix(curr_y_test, curr_y_pred).ravel()
        )

        curr_test_size = curr_tn + curr_fp + curr_fn + curr_tp
        assert(curr_test_size == len(curr_X_test)
                   and curr_test_size == len(curr_y_test))
        tns.append({
            'tn': curr_tn,
            'test_size': curr_test_size,
            'tn_rate': curr_tn/curr_test_size,
        })
        fps.append({
            'fp': curr_fp,
            'test_size': curr_test_size,
            'fp_rate': curr_fp/curr_test_size,
        })
        fns.append({
            'fn': curr_fn,
            'test_size': curr_test_size,
            'fn_rate': curr_fn/curr_test_size,
        })
        tps.append({
            'tp': curr_tp,
            'test_size': curr_test_size,
            'tp_rate': curr_tp/curr_test_size,
        })

    tns_df = pd.DataFrame(tns)
    fps_df = pd.DataFrame(fps)
    fns_df = pd.DataFrame(fns)
    tps_df = pd.DataFrame(tps)
    return (tns_df, fps_df, fns_df, tps_df)

Статистика о true negatives (файл: clear, предсказание: clear):

In [5]:
def get_worst_tn_rate(tns_df):
    return tns_df.loc[tns_df['tn_rate'].idxmax()]['tn_rate']

def get_average_tn_rate(tns_df):
    return (tns_df['tn_rate'].mean(), tns_df['tn_rate'].std()*2)

Статистика о false positives (файл: malware, предсказание: clear):

In [6]:
def get_worst_fp_rate(fps_df):
    return fps_df.loc[fps_df['fp_rate'].idxmax()]['fp_rate']

def get_average_fp_rate(fps_df):
    return (fps_df['fp_rate'].mean(), fps_df['fp_rate'].std()*2)

Статистика о false negatives (файл: clear, предсказание: malware):

In [7]:
def get_worst_fn_rate(fns_df):
    return fns_df.loc[fns_df['fn_rate'].idxmax()]['fn_rate']

def get_average_fn_rate(fns_df):
    return (fns_df['fn_rate'].mean(), fns_df['fn_rate'].std()*2)

Статистика о true positives (файл: malware, предсказание: malware):

In [8]:
def get_worst_tp_rate(tps_df):
    return tps_df.loc[tps_df['tp_rate'].idxmax()]['tp_rate']

def get_average_tp_rate(tps_df):
    return (tps_df['tp_rate'].mean(), tps_df['tp_rate'].std()*2)

Accuracy (точность предсказания):

In [9]:
def get_accuracy(tns_df, tps_df):
    accuracy = tps_df['tp_rate'].mean() + tns_df['tn_rate'].mean()
    deviation = (tps_df['tp_rate'] + tns_df['tn_rate']).std()*2
    # deviation = 2*np.sqrt(tps_df['tp_rate'].std()**2 + tns_df['tn_rate'].std()**2)
    return (accuracy, deviation)

Протестируем классификатор:

In [10]:
classifier = joblib.load(PATH_TO_CLASSIFIER)
tns_df, fps_df, fns_df, tps_df = (
    test_classifier(classifier, X_train, y_train)
)

print('TRAIN accuracy = %0.4f (+/- %0.4f)'
      % get_accuracy(tns_df, tps_df))
# print('TRAIN worst false alarm rate =', get_worst_fp_rate(fps_df))
print()
print('TRAIN fp rate =', get_average_fp_rate(fps_df))
print('TRAIN tp rate =', get_average_tp_rate(tps_df))
print('TRAIN tn rate =', get_average_tn_rate(tns_df))
print('TRAIN fn rate =', get_average_fn_rate(fns_df))

*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
*** fitted ***
TRAIN accuracy = 0.9631 (+/- 0.0041)

TRAIN fp rate = (0.014989866305210101, 0.0025116638397508968)
TRAIN tp rate = (0.48287595384806625, 0.009687347790860504)
TRAIN tn rate = (0.48025252005154917, 0.008651097570386857)
TRAIN fn rate = (0.021881659795174528, 0.004388522134734516)


In [11]:
classifier = joblib.load(PATH_TO_CLASSIFIER)
y_pred = classifier.predict(X_test)

final_test_tn, final_test_fp, final_test_fn, final_test_tp = (
    confusion_matrix(y_test, y_pred).ravel()
)

print('FINAL accuracy =', (final_test_tp + final_test_tn)/len(y_pred))
print('FINAL false alarm rate =', final_test_fp/len(y_pred))
print()
print('FINAL tp rate =', final_test_tp/len(y_pred))
print('FINAL tn rate =', final_test_tn/len(y_pred))
print('FINAL fn rate =', final_test_fn/len(y_pred))

FINAL accuracy = 0.964705227412
FINAL false alarm rate = 0.0121917274397

FINAL tp rate = 0.477592829706
FINAL tn rate = 0.487112397706
FINAL fn rate = 0.0231030451484


In [12]:
sum(y_pred)

8798.0